In [1]:
import pandas as pd
import json

# Schüler*abgängerinnen ohne Schulabschluss 2015 in Niedersachsen

### __Step1__: Abfrage über https://api-next.datengui.de/graphiql

```
{
  allRegions(page: 0, itemsPerPage: 1000) {
    regions(parent: "03", nuts: 3) {
      name
      BILABS(GES:GESAMT, BILAG2:[BILABS11, GESAMT], year:2015) {
        value
        year
      }
    }
  }
}
```

### __Step 2__: Daten manuell kopieren und in json-File speichern: "abs_15-nds.json"

### __Step 3__: Hier mit python weiterarbeiten.

In [2]:
data = json.load(open('abs_15-nds.json'))

In [3]:
data

{'data': {'allRegions': {'regions': [{'name': 'Braunschweig',
     'BILABS': [{'value': 2612, 'year': 2015}, {'value': 130, 'year': 2015}]},
    {'name': 'Salzgitter',
     'BILABS': [{'value': 1017, 'year': 2015}, {'value': 49, 'year': 2015}]},
    {'name': 'Wolfsburg',
     'BILABS': [{'value': 1447, 'year': 2015}, {'value': 41, 'year': 2015}]},
    {'name': 'Gifhorn',
     'BILABS': [{'value': 1664, 'year': 2015}, {'value': 66, 'year': 2015}]},
    {'name': 'Göttingen',
     'BILABS': [{'value': 2486, 'year': 2015}, {'value': 105, 'year': 2015}]},
    {'name': 'Goslar',
     'BILABS': [{'value': 1200, 'year': 2015}, {'value': 54, 'year': 2015}]},
    {'name': 'Helmstedt',
     'BILABS': [{'value': 831, 'year': 2015}, {'value': 73, 'year': 2015}]},
    {'name': 'Northeim',
     'BILABS': [{'value': 1571, 'year': 2015}, {'value': 71, 'year': 2015}]},
    {'name': 'Osterode am Harz',
     'BILABS': [{'value': 656, 'year': 2015}, {'value': 20, 'year': 2015}]},
    {'name': 'Peine',
    

In [4]:
df = pd.DataFrame(data["data"]["allRegions"]["regions"])

In [5]:
df.head()

,BILABS,name
0,"[{'value': 2612, 'year': 2015}, {'value': 130,...",Braunschweig
1,"[{'value': 1017, 'year': 2015}, {'value': 49, ...",Salzgitter
2,"[{'value': 1447, 'year': 2015}, {'value': 41, ...",Wolfsburg
3,"[{'value': 1664, 'year': 2015}, {'value': 66, ...",Gifhorn
4,"[{'value': 2486, 'year': 2015}, {'value': 105,...",Göttingen


In [6]:
df["BILABS"][0]

[{'value': 2612, 'year': 2015}, {'value': 130, 'year': 2015}]

In [7]:
def convert(df):
    for _, row in df.iterrows():
        for line in row['BILABS']:
            line['name'] = row['name']
            yield line

In [8]:
def convert(df):
    for _, row in df.iterrows():
        row['Gesamt'] = None
        row['Ohne Abschluss'] = None
        for line in row['BILABS']:
            line['name'] = row['name']
            if row['Gesamt'] == None:
                row['Gesamt'] = line['value']
                row['Ohne Abschluss'] = line['value']
            elif row['Gesamt'] < line['value']:
                row['Gesamt'] = line['value']
            else:
                row['Ohne Abschluss'] = line['value']
        yield row

In [9]:
df = pd.DataFrame(convert(df))
df.head()

,BILABS,name,Gesamt,Ohne Abschluss
0,"[{'value': 2612, 'year': 2015, 'name': 'Brauns...",Braunschweig,2612.0,130.0
1,"[{'value': 1017, 'year': 2015, 'name': 'Salzgi...",Salzgitter,1017.0,49.0
2,"[{'value': 1447, 'year': 2015, 'name': 'Wolfsb...",Wolfsburg,1447.0,41.0
3,"[{'value': 1664, 'year': 2015, 'name': 'Gifhor...",Gifhorn,1664.0,66.0
4,"[{'value': 2486, 'year': 2015, 'name': 'Göttin...",Göttingen,2486.0,105.0


In [10]:
df.columns

Index(['BILABS', 'name', 'Gesamt', 'Ohne Abschluss'], dtype='object')

In [16]:
df1 = df[['name', 'Gesamt', 'Ohne Abschluss']]

In [17]:
df1.head()

,name,Gesamt,Ohne Abschluss
0,Braunschweig,2612.0,130.0
1,Salzgitter,1017.0,49.0
2,Wolfsburg,1447.0,41.0
3,Gifhorn,1664.0,66.0
4,Göttingen,2486.0,105.0


In [21]:
df1['Anteil ohne Abschluss'] = (df1['Ohne Abschluss'] / df1['Gesamt'])*100

In [22]:
df1 = df1[['name', 'Anteil ohne Abschluss']]

In [23]:
df1 = df1.dropna()

In [24]:
df1.shape

(46, 2)

In [25]:
df1.to_csv('Anteil_ohne_Abschluss_15.csv')

In [26]:
df1.sort_values(by='Anteil ohne Abschluss').head(10)

,name,Anteil ohne Abschluss
21,Harburg,2.289203
2,Wolfsburg,2.833449
13,Diepholz,2.941176
8,Osterode am Harz,3.048780
24,Osterholz,3.181077
23,Lüneburg,3.188259
18,Schaumburg,3.227735
28,Uelzen,3.323558
34,Wilhelmshaven,3.629032
3,Gifhorn,3.966346


In [27]:
df1.sort_values(by='Anteil ohne Abschluss', ascending=False).head(10)

,name,Anteil ohne Abschluss
31,Emden,10.240964
6,Helmstedt,8.784597
16,Holzminden,8.532934
41,Leer,8.396533
40,Grafschaft Bentheim,7.577640
22,Lüchow-Dannenberg,7.074570
36,Aurich,6.976744
15,Hildesheim,6.894350
26,Heidekreis,6.760910
30,Delmenhorst,6.524184


In [28]:
df1

,name,Anteil ohne Abschluss
0,Braunschweig,4.977029
1,Salzgitter,4.818092
2,Wolfsburg,2.833449
3,Gifhorn,3.966346
4,Göttingen,4.223652
5,Goslar,4.500000
6,Helmstedt,8.784597
7,Northeim,4.519414
8,Osterode am Harz,3.048780
9,Peine,4.686469
